In [ ]:
#This program generates Gaussian filtered ratemaps for each cell for a session
#The input is the interpolated data from the interpolation program
#The output is a Gaussian filtered ratemap for every cell in the session
#It also plots one cell!

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import os
import math
from scipy.special import erf
from scipy import signal
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

#this is the location of the csv file of the combined Ca imaging and behavioural data for the session
path= r'/Users/insert_file_path_here'
Ca_data='session1_interpolated_data_filename.csv'
Ca_data2='session2_interpolated_data_filename.csv'
Ca_data3='session3_interpolated_data_filename.csv'
Ca_data4='session4_interpolated_data_filename.csv'

#ouput file name
newpath = path + r'/output_filename' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
testName = 'Insert_Mouse_ID_here'

### Need to change "threshold_time"/"Speed"/"Spike_Threthold" depending on parameters you applied on place cell identification

In [ ]:
#define variables and file locations

#Gaussian standadrd deviation in grid squares; 
Std_dev=2 #bins not cm

#Timestep size
Delta_time=0.049962

#define number of cells detected (ignore - replaced later)
N_cells=700

#define number of bins/gridcells
Gridcells_x=16
Gridcells_y=16
N_bins=Gridcells_x*Gridcells_y

#defines whether the data is pruned to remove the times the mouse doesn't move (1=pruning on, 0=pruning off)
pruning =1
#IF using amplitude rather than event for rate activity,
amplitude = 0

#defines the threshold for counting events
Spike_threshold=5

#defines the threshold velocity for pruning 
#if the velocity os below this, it is pruned when pruning = True
speed_velocity = 1
#if the velocity is below this, it is pruned when pruning = True
Threshold_velocity=0.01

#threshold time in grid square (set to zero when speed thresholding is used)
#if the time in a grid square is below this, the activity in teh grid square is zeroed 
threshold_time=0


#defines the threshold velocity for pruning (in m/s?)
#If using n% of average speed,
average_speed = 0
speed_fraction=0.1

# function definitions begin here

In [ ]:
#defines the activity map function. 
#Takes as input raw cell and behaioural data, outputs the activity per bin per cell
def activity_map(input_df):
    df=input_df.drop(['Time','X center','Y center','Speed'], axis=1)
    df=df[df>0].groupby('Grid no.').count()
    return(df)

In [ ]:
#alternative for summing amplitudes
if amplitude ==1:
    def activity_map(input_df):
        df=input_df.drop(['Time','X center','Y center','Speed'], axis=1)
        df=df[df>0].groupby('Grid no.').sum()
        return(df)

In [ ]:
#define velocity pruning function 
#prunes data with velocity less than threshold velocity from the input data
#prunes spikes with an amplitude less than spike threshold
if speed_velocity==1:
    def prune (df):
        #prune non-moving times
        pruned_velocity=df.drop(df[df.Speed < Threshold_velocity].index).reset_index(drop=True)
        #set spikes with size less than a threshold to zero  
        cell_data_only=pruned_velocity.iloc[:,5:N_cells+5] #get cell data only
        behavioural_data_only=pruned_velocity.iloc[:,0:5] #get behavioural data only
        cell_data_only=cell_data_only.mask(cell_data_only<Spike_threshold, 0) #mask cell data
        pruned_data=pd.concat([behavioural_data_only, cell_data_only], axis = 1) #recombine
        
        return(pruned_data)

In [ ]:
#define velocity pruning function by using average speed 
#prunes data with velocity less than threshold velocity from the input data
#prunes spikes with an amplitude less than spike threshold
if average_speed==1:
    def prune (df):
        average_speed= df['Speed'].mean()
        thresh_speed=average_speed*speed_fraction
        pruned_velocity=df.drop(df[df.Speed < thresh_speed].index).reset_index(drop=True)
        #set spikes with size less than a threshold to zero  
        cell_data_only=pruned_velocity.iloc[:,5:N_cells+5] #get cell data only
        behavioural_data_only=pruned_velocity.iloc[:,0:5] #get behavioural data only
        cell_data_only=cell_data_only.mask(cell_data_only<Spike_threshold, 0) #mask cell data
        pruned_data=pd.concat([behavioural_data_only, cell_data_only], axis = 1) #recombine
        
        return(pruned_data)

# main program begins here with session 1

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#prune data, is pruning variable is set
if pruning ==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)
interpolated_data_for_session

In [ ]:
#determine number of cells detected
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#calculate the overall occupancy time per bin and probability of being in each bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map 
cell_activations_per_bin=activity_map(interpolated_data_for_session)
cell_activations_per_bin

In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)

In [ ]:
#calulate raw rate map by dividing activity map by occupancy time
rate_maps = cell_activations_per_bin.div(occupancy_time['Duration'], axis=0).fillna(0)
rate_maps

In [ ]:
#write to file
rate_maps.to_csv(f'{newpath}/{testName}D1S1_ratemaps.csv', index=True)

In [ ]:
#Applies a gaussian filter to the rate map

#initialises the dataframe for the filtered ratemaps 
filtered_rate_map_df=pd.DataFrame(np.random.randint(0,256,size=(256, 1)), columns=['Delete'])

#initialises the numpy array for individual ratemaps 
cell_rate_map=np.zeros((Gridcells_x,Gridcells_y))

#Gaussian filters every column
for column in rate_maps:
    #the next two for loops convert the pandas column into a N by N numpy array of teh ratemap
    for i in range(Gridcells_x):
        for j in range(Gridcells_y):
            index=i+16*j+1
            cell_rate_map[j,i]=rate_maps.at[index,column]
    filtered_rate_map=gaussian_filter(cell_rate_map, sigma=Std_dev) #applies Gaussian filter to the rate map
    intermediate_array=filtered_rate_map.flatten() #flattens array
    filtered_rate_map_column=pd.DataFrame(intermediate_array, columns=[column]) #converts array to pandas column
    filtered_rate_map_df=pd.concat([filtered_rate_map_df, filtered_rate_map_column], axis=1) #concatenates the pandas column
    
filtered_rate_map_df=filtered_rate_map_df.drop(['Delete'], axis=1) #deletes the initialisation column

filtered_rate_map_df

In [ ]:
#write filtered ratemap to file
filtered_rate_map_df.to_csv(f'{newpath}/{testName}D1S1_gaussian_filtered_ratemaps.csv', index=True)

# From here, session 2

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data2))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#prune data, is pruning variable is set
if pruning ==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)

In [ ]:
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#calculate the overall occupancy per bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map 
cell_activations_per_bin=activity_map(interpolated_data_for_session)
cell_activations_per_bin

In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)

In [ ]:
#calulate rate map by dividing activity map by occupancy time
rate_maps = cell_activations_per_bin.div(occupancy_time['Duration'], axis=0).fillna(0)
rate_maps

In [ ]:
#write to file
rate_maps.to_csv(f'{newpath}/{testName}D1S2_ratemaps.csv', index=True)

In [ ]:
#Applies a gaussian filter to the rate map

#initialises the dataframe for the filtered ratemaps 
filtered_rate_map_df=pd.DataFrame(np.random.randint(0,256,size=(256, 1)), columns=['Delete'])

#initialises the numpy array for individual ratemaps 
cell_rate_map=np.zeros((Gridcells_x,Gridcells_y))

#Gaussian filters every column
for column in rate_maps:
    #the next two for loops convert the pandas column into a N by N numpy array of teh ratemap
    for i in range(Gridcells_x):
        for j in range(Gridcells_y):
            index=i+16*j+1
            cell_rate_map[j,i]=rate_maps.at[index,column]
    filtered_rate_map=gaussian_filter(cell_rate_map, sigma=Std_dev) #applies Gaussian filter to the rate map
    intermediate_array=filtered_rate_map.flatten() #flattens array
    filtered_rate_map_column=pd.DataFrame(intermediate_array, columns=[column]) #converts array to pandas column
    filtered_rate_map_df=pd.concat([filtered_rate_map_df, filtered_rate_map_column], axis=1) #concatenates the pandas column
    
filtered_rate_map_df=filtered_rate_map_df.drop(['Delete'], axis=1) #deletes the initialisation column

filtered_rate_map_df

In [ ]:
#write to file
filtered_rate_map_df.to_csv(f'{newpath}/{testName}D1S2_gaussian_filtered_ratemaps.csv', index=True)

# From here, session 3

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data3))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#prune data, is pruning variable is set
if pruning ==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)

In [ ]:
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#calculate the overall occupancy per bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map 
cell_activations_per_bin=activity_map(interpolated_data_for_session)
cell_activations_per_bin

In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)

In [ ]:
#calulate rate map by dividing activity map by occupancy time
rate_maps = cell_activations_per_bin.div(occupancy_time['Duration'], axis=0).fillna(0)
rate_maps

In [ ]:
#write to file
rate_maps.to_csv(f'{newpath}/{testName}D2S1_ratemaps.csv', index=True)

In [ ]:
#Applies a gaussian filter to the rate map

#initialises the dataframe for the filtered ratemaps 
filtered_rate_map_df=pd.DataFrame(np.random.randint(0,256,size=(256, 1)), columns=['Delete'])

#initialises the numpy array for individual ratemaps 
cell_rate_map=np.zeros((Gridcells_x,Gridcells_y))

#Gaussian filters every column
for column in rate_maps:
    #the next two for loops convert the pandas column into a N by N numpy array of teh ratemap
    for i in range(Gridcells_x):
        for j in range(Gridcells_y):
            index=i+16*j+1
            cell_rate_map[j,i]=rate_maps.at[index,column]
    filtered_rate_map=gaussian_filter(cell_rate_map, sigma=Std_dev) #applies Gaussian filter to the rate map
    intermediate_array=filtered_rate_map.flatten() #flattens array
    filtered_rate_map_column=pd.DataFrame(intermediate_array, columns=[column]) #converts array to pandas column
    filtered_rate_map_df=pd.concat([filtered_rate_map_df, filtered_rate_map_column], axis=1) #concatenates the pandas column
    
filtered_rate_map_df=filtered_rate_map_df.drop(['Delete'], axis=1) #deletes the initialisation column

filtered_rate_map_df

In [ ]:
#write to file
filtered_rate_map_df.to_csv(f'{newpath}/{testName}D2S1_gaussian_filtered_ratemaps.csv', index=True)

# From here, session 4

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data4))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#prune data, is pruning variable is set
if pruning ==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)

In [ ]:
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#calculate the overall occupancy per bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map 
cell_activations_per_bin=activity_map(interpolated_data_for_session)
cell_activations_per_bin

In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)

In [ ]:
#calulate rate map by dividing activity map by occupancy time
rate_maps = cell_activations_per_bin.div(occupancy_time['Duration'], axis=0).fillna(0)
rate_maps

In [ ]:
#write to file
rate_maps.to_csv(f'{newpath}/{testName}D2S2_ratemaps.csv', index=True)

In [ ]:
#Applies a gaussian filter to the rate map

#initialises the dataframe for the filtered ratemaps 
filtered_rate_map_df=pd.DataFrame(np.random.randint(0,256,size=(256, 1)), columns=['Delete'])

#initialises the numpy array for individual ratemaps 
cell_rate_map=np.zeros((Gridcells_x,Gridcells_y))

#Gaussian filters every column
for column in rate_maps:
    #the next two for loops convert the pandas column into a N by N numpy array of teh ratemap
    for i in range(Gridcells_x):
        for j in range(Gridcells_y):
            index=i+16*j+1
            cell_rate_map[j,i]=rate_maps.at[index,column]
    filtered_rate_map=gaussian_filter(cell_rate_map, sigma=Std_dev) #applies Gaussian filter to the rate map
    intermediate_array=filtered_rate_map.flatten() #flattens array
    filtered_rate_map_column=pd.DataFrame(intermediate_array, columns=[column]) #converts array to pandas column
    filtered_rate_map_df=pd.concat([filtered_rate_map_df, filtered_rate_map_column], axis=1) #concatenates the pandas column
    
filtered_rate_map_df=filtered_rate_map_df.drop(['Delete'], axis=1) #deletes the initialisation column

filtered_rate_map_df

In [ ]:
#write to file
filtered_rate_map_df.to_csv(f'{newpath}/{testName}D2S2_gaussian_filtered_ratemaps.csv', index=True)